In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data **Reading****

In [0]:
df= spark.read.format("parquet").load("abfss://bronze@databricksadlsete.dfs.core.windows.net/products")

In [0]:
df.display()

In [0]:
df=df.drop("_rescued_data")


In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_ete_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
RETURN p_price * 0.9


In [0]:
%sql
select product_id,price,databricks_ete_cata.bronze.discount_func(price) as discounted_price
from products

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_ete_cata.bronze.Upper_Func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
drop function if exists databricks_ete_cata.bronze.upper

In [0]:
%sql
select product_id,brand,databricks_ete_cata.bronze.Upper_Func(brand) as brand_upper
from products

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@databricksadlsete.dfs.core.windows.net/products")\
    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_ete_cata.silver.Products_silver
USING DELTA
LOCATION 'abfss://silver@databricksadlsete.dfs.core.windows.net/products'

In [0]:
#%sql
#select * from databricks_ete_cata.silver.Products_silver